# Confirmados - gráficos
> Gráficos de acumulado de infectados confirmados.

- toc: false 
- badges: false
- comments: false

## Total

In [1]:
#collapse

import requests
import pandas as pd
from io import BytesIO
import numpy as np
import altair as alt

# get data
response = requests.get('https://docs.google.com/spreadsheet/ccc?key=1FwOXbkWeuS1LHrKxhSfepGtsgQ6iIEQZmWig_JKo8z0&output=csv')
assert response.status_code == 200, 'Wrong status code'
data = response.content
df = pd.read_csv(BytesIO(data), parse_dates=['Fecha'], dayfirst=True)

# Create a selection that chooses the nearest point & selects based on x-value
source = df
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Total confirmados'], empty='none')

# The basic line
line = alt.Chart(source).mark_line(interpolate='basis').encode(
    x='monthdate(Fecha):O',
    y='Total confirmados:Q',
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(source).mark_point().encode(
    x='monthdate(Fecha):O',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Total confirmados:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(source).mark_rule(color='gray').encode(
    x='monthdate(Fecha):O',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=800, height=400
)

alt.LayerChart(...)

## Por comunidad autónoma

In [4]:
#collapse

import requests
import pandas as pd
from io import BytesIO
import numpy as np
import altair as alt

# data download
response = requests.get('https://docs.google.com/spreadsheet/ccc?key=1FwOXbkWeuS1LHrKxhSfepGtsgQ6iIEQZmWig_JKo8z0&output=csv')
assert response.status_code == 200, 'Wrong status code'
data = response.content
df = pd.read_csv(BytesIO(data), parse_dates=['Fecha'], dayfirst=True)

# plot
cols = list(df.columns)
cols.remove('Total confirmados')
cols.remove('Fecha')
df = df.set_index('Fecha')[cols]
source = df.reset_index().melt('Fecha', var_name='category', value_name='Confirmados')
# plot
selection = alt.selection_multi(fields=['category'], bind='legend')

alt.Chart(source).mark_line(interpolate='basis').encode(
    alt.X('monthdate(Fecha):O', axis=alt.Axis()),
    alt.Y('Confirmados:Q'),
    alt.Color('category:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).properties(
    width=800, height=400
).add_selection(
    selection
)

alt.Chart(...)